<a href="https://colab.research.google.com/github/igohawk/machine-learning-project/blob/master/Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving CN.zip to CN.zip
Saving EN.zip to EN.zip
Saving FR.zip to FR.zip
Saving SG.zip to SG.zip
User uploaded file "CN.zip" with length 555571 bytes
User uploaded file "EN.zip" with length 46015 bytes
User uploaded file "FR.zip" with length 80542 bytes
User uploaded file "SG.zip" with length 1282785 bytes


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('./CN.zip', 'r')
zip_ref.extractall('./CN')
zip_ref.close()

In [0]:
def data_tuple(file):

    data = []
    lst = []
    with open(file, "r") as f:
        for line in f:
            if line == "\n":
              data.append(lst)
              lst = []
            
            else:
              lines = line.replace("\n",'').split(" ")
              lst.append(tuple(lines))
            
    return data


  
data_tuple('SG/train')

In [0]:
# Write a function that estimates the emission parameters from the training set using MLE (maximum
# likelihood estimation):
# e(x|y) = Count(y -> x)/Count(y)
# with key <word, tag>

def emission_parameter(data):
    
    tag_count = {}
    match_count = {}
    
    for lst in data:         
        for wordTuples in lst:
            
            tag = wordTuples[1]
            
            tag_count[tag] = tag_count.get(tag, 0) + 1
            match_count[wordTuples] = match_count.get(wordTuples, 0) + 1
            
      
    em = {i: match_count[i]/tag_count[i[1]] for i in match_count}   
    return tag_count, em
  
emission_parameter(data_tuple('SG/train'))



In [0]:
# During the testing phase, if the word does not appear in the training set, we replace that word with the
# special word token #UNK#

# Set k to 1:

def smoothing_em(data, k):

    new_data = []
    new_tweets = []
  
    word_count = {}
    
    for lst in data:
        for wordTuples in lst:
          
            word = wordTuples[0]
            tag = wordTuples[1]
            
            word_count[word] = word_count.get(word, 0) + 1

            if word_count[word] < k:
              
               new_tweets.append(("#UNK#",tag))
                
            else:
              
                new_tweets.append(wordTuples)
                
        new_data.append(new_tweets)
        new_tweets = []
        
    return new_data,word_count

emission_parameter(smoothing_em(data_tuple('SG/train'),1)[0])



In [109]:
def sentiment_analysis(word, tags, em):
    
    pre_set = (word, "O") # if word not in train set, set tag to 0
    max_em = 0
    
    for tag in tags:
        if (word, tag) in em.keys():
            if em[(word, tag)] > max_em: # to select the max em value
                pre_set = (word, tag)
                max_em = em[(word, tag)]
    return pre_set



word = "hell"
tags = emission_parameter(data_tuple('SG/train'))[0].keys()
em = emission_parameter(smoothing_em(data_tuple('SG/train'),1)[0])[1]

sentiment_analysis(word,tags,em)

('hell', 'B-negative')

In [0]:
# Implement a simple sequence labeling system that produces the tag

# for each word x in the sequence.

def tag_predict(language, q, words, tags, em):
   
    input_file = language + "/dev.in"
    output_file = language + "/dev.p"+ q +".out"
    
    with open(input_file, "r") as f:
        with open(output_file, "w") as output:
            for line in f:
                if line =="\n":
                    output.write("\n")
                    continue
                    
                if line.replace("\n","") in words:
                    test_word = line.replace("\n","")
                    pre_set = sentiment_analysis(test_word, tags, em)
                    output.write(" ".join(pre_set)+"\n")
                else:
                    output.write("#UNK# O\n")
        #files.download(output_file)
        
# words = smoothing_em(data_tuple(tf),1)[1].keys()
# tags = emission_parameter(data_tuple(tf))[0].keys()
# em = emission_parameter(smoothing_em(data_tuple(tf),1)[0])[1]
# tf = language + "/train"
# language = "SG"

# tag_predict(language,"2",words,tags,em)                    
                    


In [0]:
# Evaluation

def get_entities(observed, sep=' ', output_col=1):

    example = 0
    word_index = 0
    entity = []
    last_ne = 'O'
    last_sent = ''
    last_entity = []

    observations = {}
    observations[example] = []

    for line in observed:
        line = line.strip()
        if line.startswith('##'):
            continue
        elif len(line) == 0:
            if entity:
                observations[example].append(list(entity))
                entity = []

            example += 1
            observations[example] = []
            word_index = 0
            last_ne = 'O'
            continue

        split_line = line.split(sep)
        value = split_line[output_col]
        ne = value[0]
        sent = value[2:]

        last_entity = []

        # check if it is start of entity
        if ne == 'B' or (ne == 'I' and last_ne == 'O') or \
                (ne == 'I' and last_ne != 'O' and last_sent != sent):
            if entity:
                last_entity = entity
            entity = [sent]
            entity.append(word_index)
        elif ne == 'I':
            entity.append(word_index)
        elif ne == 'O':
            if last_ne == 'B' or last_ne == 'I':
                last_entity = entity
            entity = []

        if last_entity:
            observations[example].append(list(last_entity))
            last_entity = []

        last_ne = ne
        last_sent = sent
        word_index += 1

    if entity:
        observations[example].append(list(entity))

    return observations


def compare_result(observed, predicted):
    
    total_observed = 0
    total_predicted = 0
    correct = {'Entities': 0, 'Sentiment': 0}

    for example in observed:
        observed_instance = observed[example]
        predicted_instance = predicted[example]
        total_observed += len(observed_instance)
        total_predicted += len(predicted_instance)

        for span in predicted_instance:
            span_sent = span[0]
            span_ne = (span[1], len(span) - 1)

            for observed_span in observed_instance:
                sent = observed_span[0]
                ne = (observed_span[1], len(observed_span) - 1)

                if span_ne == ne:
                    correct['Entities'] += 1
                    if span_sent == sent:
                        correct['Sentiment'] += 1

    print('Entities:')
    print('In gold-standard outputs: %d' % total_observed)
    print('In prediction outputs: %d' % total_predicted)
    print()
    for t in ('Entities', 'Sentiment'):
        prec = correct[t] / total_predicted
        recl = correct[t] / total_observed
        try:
            f = 2 * prec * recl / (prec + recl)
        except ZeroDivisionError:
            f = 0
        print('%s Correct: %d' % (t, correct[t]))
        print('%s F score: %.4f (Precision: %.4f, Recall: %.4f)' % (t, f, prec, recl))
        print()


In [158]:
for lan in ["CN", "EN", "SG", "FR"]:

    data = data_tuple(lan + "/train")
    new_data = smoothing_em(data,1)[0]
    words = smoothing_em(data,1)[1].keys()
    tags = emission_parameter(data)[0].keys()
    smoothed_em = emission_parameter(new_data)[2]
    
    tag_predict(lan,"2",words, tags, smoothed_em)
    
    with open(lan + '/dev.p2.out' ) as f:
        pred = get_entities(f)
    with open(lan + '/dev.out' ) as f:
        gold = get_entities(f)
        
    print(lan)
    compare_result(gold, pred)


CN
Entities:
In gold-standard outputs: 1081
In prediction outputs: 3963

Entities Correct: 485
Entities F score: 0.1923 (Precision: 0.1224, Recall: 0.4487)

Sentiment Correct: 347
Sentiment F score: 0.1376 (Precision: 0.0876, Recall: 0.3210)

EN
Entities:
In gold-standard outputs: 802
In prediction outputs: 770

Entities Correct: 489
Entities F score: 0.6221 (Precision: 0.6351, Recall: 0.6097)

Sentiment Correct: 448
Sentiment F score: 0.5700 (Precision: 0.5818, Recall: 0.5586)

SG
Entities:
In gold-standard outputs: 4092
In prediction outputs: 9283

Entities Correct: 1999
Entities F score: 0.2989 (Precision: 0.2153, Recall: 0.4885)

Sentiment Correct: 1179
Sentiment F score: 0.1763 (Precision: 0.1270, Recall: 0.2881)

FR
Entities:
In gold-standard outputs: 238
In prediction outputs: 826

Entities Correct: 175
Entities F score: 0.3289 (Precision: 0.2119, Recall: 0.7353)

Sentiment Correct: 78
Sentiment F score: 0.1466 (Precision: 0.0944, Recall: 0.3277)

